<img src='http://hilpisch.com/taim_logo.png' width="350px" align="right">

# Artificial Intelligence in Finance

## Superintelligence

Dr Yves J Hilpisch | The AI Machine

http://aimachine.io | http://twitter.com/dyjh

In [ ]:
!git clone https://github.com/tpq-classes/ai_in_finance_book.git
import sys
sys.path.append('ai_in_finance_book')


In [ ]:
import gymnasium as gym
import numpy as np
import pandas as pd
np.random.seed(100)

In [ ]:
env = gym.make('CartPole-v1')

In [ ]:
env.action_space.seed(100)

In [ ]:
action_size = env.action_space.n
action_size

In [ ]:
[env.action_space.sample() for _ in range(10)]

In [ ]:
state_size = env.observation_space.shape[0]
state_size

In [ ]:
state = env.reset()
state  # [cart position, cart velocity, pole angle, pole angular velocity]

In [ ]:
state, reward, done, _, _ = env.step(env.action_space.sample())
state, reward, done, _

In [ ]:
%%time
data = pd.DataFrame()
state = env.reset()
length = []
for run in range(25000):
    done = False
    prev_state = env.reset()[0]
    treward = 1
    results = []
    while not done:
        action = env.action_space.sample()
        state, reward, done, trunc, info = env.step(action)
        results.append({'s1': prev_state[0], 's2': prev_state[1],
                        's3': prev_state[2], 's4': prev_state[3],
                        'a': action, 'r': reward})
        treward += reward if not done else 0
        prev_state = state
    if treward >= 110:
        data = pd.concat((data, pd.DataFrame(results)))
        length.append(treward)

In [ ]:
np.array(length).mean()

In [ ]:
data.info()

In [ ]:
data.tail()

In [ ]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

In [ ]:
from pylab import plt
plt.style.use('seaborn-v0_8')
%matplotlib inline

In [ ]:
import tensorflow as tf
from tensorflow import keras
tf.get_logger().setLevel('WARN')
tf.random.set_seed(100)

In [ ]:
#disable_eager_execution()

In [ ]:
from keras.layers import Dense
from keras.models import Sequential

In [ ]:
model = Sequential()
model.add(Dense(64, activation='relu'))
                #input_dim=env.observation_space.shape[0]))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['acc'])

In [ ]:
%%time
model.fit(data[['s1', 's2', 's3', 's4']], data['a'],
          epochs=25, verbose=False, validation_split=0.2)

In [ ]:
res = pd.DataFrame(model.history.history)
res.tail(3)

In [ ]:
res.plot(figsize=(10, 6), style='--');

In [ ]:
def epoch():
    print('|', end='')
    done = False
    state = env.reset()[0]
    treward = 1
    while not done:
        action = np.where(model.predict(np.atleast_2d(state), verbose=0)[0][0] > 0.5, 1, 0)
        state, reward, done, trunc, info = env.step(action)
        treward += reward if not done else 0
    return treward

In [ ]:
epoch()

In [ ]:
%%time
res = np.array([epoch() for _ in range(100)])
print()
res

In [ ]:
res.mean()

<img src='http://hilpisch.com/taim_logo.png' width="350px" align="right">

<br><br><br><a href="http://tpq.io" target="_blank">http://tpq.io</a> | <a href="http://twitter.com/dyjh" target="_blank">@dyjh</a> | <a href="mailto:ai@tpq.io">ai@tpq.io</a>